# Testing VGG16 network

In this notebook I use the data explored and processed in notebooks 1-3 to see how well the pretrained VGG16 model works as a classifier.

In [1]:
import os
import time
import pickle

import numpy as np

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import metrics

#TODO implement this function
#from cars_utils import plot_metrics

Using TensorFlow backend.


### BASIC  IMAGE, AND BATCH SETUP

In [26]:
# Image dimensions:
IMG_WIDTH, IMG_HEIGHT = 100*2, 75*2

BASE = os.getcwd()
SAVE_DIR = os.path.join(BASE, 'data', 'VGG16Av2')
BASE = os.path.join(BASE, 'data', 'cars_dataset', 'keras')

### Settings for full training
# TRAIN_DATA_DIR = os.path.join(BASE, 'train')
# VALIDATION_DATA_DIR = os.path.join(BASE, 'validation')
# NB_CLASSES = 196
# NB_TRAIN_SAMPLES = 11329
# NB_VALIDATION_SAMPLES = 2428
# BATCH_SIZE = 64

### Settings for fast checking
TRAIN_DATA_DIR = os.path.join(BASE, 'fast', 'train')
VALIDATION_DATA_DIR = os.path.join(BASE, 'fast', 'validation')
NB_CLASSES = 196
NB_TRAIN_SAMPLES = NB_CLASSES*2
NB_VALIDATION_SAMPLES = NB_CLASSES
BATCH_SIZE = 4

if K.image_data_format() == 'channels_first':
    INPUT_SHAPE = (3, IMG_WIDTH, IMG_HEIGHT)
else:
    INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)

In [27]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=INPUT_SHAPE)

In [28]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(512*4, activation='relu'))
model.add(Dense(512*4, activation='relu'))
model.add(Dense(NB_CLASSES, activation='softmax'))

conv_base.trainable = True
print('This is the number of trainable weights '
      'before freezing the conv base:', len(model.trainable_weights))
conv_base.trainable = False
print('This is the number of trainable weights '
      'after freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base: 32
This is the number of trainable weights after freezing the conv base: 6


In [29]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1/255)

In [30]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 392 images belonging to 196 classes.
Found 196 images belonging to 196 classes.


Define custom metrics to measure

In [16]:
def top_3_acc(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=3)

In [32]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[metrics.categorical_accuracy, top_3_acc])

In [33]:
EPOCHS = 1
start_time = time.time()
VGG16_0_1_summary = model.fit_generator(train_generator,
                                   steps_per_epoch=NB_TRAIN_SAMPLES // BATCH_SIZE,
                                   epochs=EPOCHS,
                                   validation_data=validation_generator,
                                   validation_steps=NB_VALIDATION_SAMPLES // BATCH_SIZE)
end_time = time.time()

print('\ntraing took: {} minutes' .format(round(end_time - start_time, 0)/60))

Epoch 1/1
98/98 [==============================] - 624s 6s/step - loss: 15.9624 - categorical_accuracy: 0.0026 - top_3_acc: 0.9796 - val_loss: 16.0359 - val_categorical_accuracy: 0.0051 - val_top_3_acc: 1.0000

traing took: 10.416666666666666 minutes
